In [1]:
!pip install -U spacy
!pip install scispacy
#!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_core_sci_lg-0.4.0.tar.gz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.2 MB 5.6 MB/s 
     |████████████████████████████████| 10.1 MB 46.6 MB/s 
     |████████████████████████████████| 181 kB 56.6 MB/s 
     |████████████████████████████████| 660 kB 57.7 MB/s 
     |████████████████████████████████| 42 kB 1.4 MB/s 
     |████████████████████████████████| 457 kB 52.2 MB/s 
     |████████████████████████████████| 58 kB 5.2 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.2.0
    Uninstalling typing-extensions-4.2.0:
      Successfully uninstalled typing-extensions-4.2.0
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 44 kB 1.9 MB/s 
     |████████████████████████████████| 71 kB 4.9 MB/s 
     |████████████████████████████████| 13.5 MB 19.0 MB/s 
     |████████████████████████████████| 6.0 MB 27.8 MB/s 
     |████████████████████████████████| 188 kB 50.9 MB/s 
  Attempting uninstall: spacy
    Found existing installation: spacy 3.3.0
    Uninstalling spacy-3.3.0:
      Successfully uninstalled spacy-3.3.0


In [2]:
import pandas as pd
import numpy as np
import re
import spacy
import scispacy
from spacy.matcher import Matcher

In [3]:
from spacy.tokens.doc import Doc
from spacy.tokens.token import Token
from spacy.tokens.span import Span
from spacy.lang.en import English

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
from zipfile import ZipFile
with ZipFile('/content/drive/MyDrive/Colab Notebooks/THESIS/en_gena_sm.zip', 'r') as file:
  file.extractall()

gena_nlp = spacy.load("en_gena_sm")
gena_nlp.pipeline

/usr/local/lib/python3.7/dist-packages/spacy/util.py:833: UserWarning: [W095] Model 'en_ner_bc5cdr_md' (0.4.0) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.2.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x7fd4100a5f30>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x7fd40fed3050>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x7fd40fe65960>),
 ('lemmatizer',
  <spacy.lang.en.lemmatizer.EnglishLemmatizer at 0x7fd40fe6fb90>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x7fd40fed70d0>),
 ('entity_ruler', <spacy.pipeline.entityruler.EntityRuler at 0x7fd40fe95b40>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x7fd40fed7450>)]

In [6]:
# Common Patterns Matching
# NOUN_PHRASE = [ {'POS': 'DET', 'OP': '?'}, {'POS': {'IN': ['ADJ', 'NOUN', 'PRON', 'PROPN', 'NUM', 'CCONJ', 'PART']}, 'OP': '+'},
#                 {'POS': 'VERB', 'DEP': {'IN': ['amod', 'acl']}, 'OP': '*'}, {'POS': {'IN': ['ADJ', 'NOUN', 'PRON', 'PROPN', 'PART', 'NUM']}, 'OP': '*'} ]

NOUN_PHRASE = [ {'POS': 'DET', 'OP': '?'}, {'POS': 'VERB', 'DEP': {'IN': ['amod', 'acl']}, 'OP': '*'}, {'POS': {'IN': ['ADJ', 'NOUN', 'PRON', 'PROPN', 'NUM', 'PART', 'CCONJ']}, 'OP': '+'}]

NO_CONJ_NOUN_PHRASE = [ {'POS': 'DET', 'OP': '?'}, {'POS': {'IN': ['ADJ', 'NOUN', 'PRON', 'PROPN', 'NUM', 'PART']}, 'OP': '+'},
              {'POS': 'VERB', 'DEP': {'IN': ['amod', 'acl']}, 'OP': '*'}, {'POS': {'IN': ['ADJ', 'NOUN', 'PRON', 'PROPN', 'PART', 'NUM']}, 'OP': '*'} ]

In [7]:
# preprocessing sentence

def is_figured_sentence(sentence:str)->bool:
  """
  Check if a sentence is a figured sentence - which talk about statistic
  """
  stat_regex = re.compile(r".*(p.*value)|(\d+\%)|(\=.*\d+).*")
  if len(stat_regex.findall(sentence)) > 0:
    return True
  return False

def process_sentence(sentence:str)->str:
  """
  Pre-processing a sentence before extract its clauses or any after-processing.
  """
  sentence = re.sub(r"[,]*[ ]*but not only[,]*", " ", sentence)
  sentence = re.sub(r"[,]*[ ]*[Nn]ot only", " ", sentence)
  sentence = re.sub(r"[,]*[ ]but also", " and ", sentence)
  sentence = re.sub(r"[Bb]oth", " ", sentence)
  sentence = re.sub(r" +- +", r"-", sentence)
  sentence = re.sub(r" +", r" ", sentence)
  return sentence.strip()
sentence = "Acute administration of the adenosine A2A receptor antagonists CSC or MSX-3 at the highest doses tested (5 and 1.25mg/kg, respectively) significantly reduces haloperidol-induced catalepsy."
is_figured_sentence(sentence), process_sentence(sentence=sentence)

(False,
 'Acute administration of the adenosine A2A receptor antagonists CSC or MSX-3 at the highest doses tested (5 and 1.25mg/kg, respectively) significantly reduces haloperidol-induced catalepsy.')

## Splitting a sentence into clauses

In [8]:
# Noun chunks processing
def has_negative_conj(noun:Token)->bool:
  """
  Check if a noun has the negative conjunction
  Example: A, but not B, is somthing.
  """
  for child in noun.children:
    if child.dep_ == 'cc' and child.text in ['not']:
      return True
  return False

def is_negative_nmod(noun:Token)->bool:
  """
  Check if a noun has the negative modify
  Example: A, without B, is something.
  """
  for child in noun.children:
    if child.dep_ in ['case', 'mark'] and child.text in ['without', 'no', 'neither', 'except']:
      return True
    if child.dep_ in ['det', 'neg'] and child.text in ['any', 'no']:
      return True
  return False

def noun_chunks(noun:Token, chunks_for:str='subj')->str:
  """
  Get chunks from a given noun.
  """
  from_noun_dep = ['compound', 'amod', 'case', 'mark', 'dep', 'nummod']
  if chunks_for == 'obj':
    conj_dep = ['conj', 'cc']
    from_noun_dep += ['punct', 'appos']
  else:
    conj_dep = []

  nmod_dep = ['nmod', 'nmod:poss']
  children = [child for child in noun.children if child.dep_ in from_noun_dep + ([] if (has_negative_conj(noun=noun)) else conj_dep)]
  children += [child for child in noun.children if child.dep_ in nmod_dep and not is_negative_nmod(child)]

  idx = 0
  while idx < len(children):
    child = children[idx]
    tmp_children = [sub_child for sub_child in child.children if sub_child.dep_ in from_noun_dep + ['mwe'] + ([] if (has_negative_conj(noun=noun)) else conj_dep)]
    tmp_children += [sub_child for sub_child in child.children if sub_child.dep_ in nmod_dep and not is_negative_nmod(sub_child)]
    if len(tmp_children) > 0:
      children += tmp_children
    idx += 1
  return (' '.join([element.text for element in sorted(children + [noun], key=lambda child: child.idx)]).replace(" 's", "'s"), noun.idx)

def is_meaning_chunk(doc:Doc, chunk:str):
  """
  Check if a given chunk is a meaning chunk in Doc.
  """
  for ent in doc.ents:
    if ent.text in chunk:
      return True
  return False

In [9]:
# Subject processing
def find_all_subj(nlp:English, sentence:str)->list:
  """
  Find all the available subjects in a given sentence.
  """
  doc = nlp(sentence)
  all_chunks_subj = []
  appos_list = []
  for token in doc:
    if token.dep_ in ['nsubj', 'nsubjpass']:
      if not has_negative_conj(noun=token):
        can_have_conj = ['conj']
      else:
        can_have_conj = []

      chunk_subj = noun_chunks(token)
      if is_meaning_chunk(doc=doc, chunk=chunk_subj[0]):
        all_chunks_subj.append(chunk_subj)
      for child in token.children:
        if child.dep_ in ['nsubj', 'appos', 'acl:relcl'] + can_have_conj:
          child_subj = noun_chunks(child)
          if is_meaning_chunk(doc=doc, chunk=child_subj[0]):
            all_chunks_subj.append(child_subj)

        elif child.dep_ in ['nmod']:
          if not has_negative_conj(noun=child):
            can_have_conj = ['conj']
          else:
            can_have_conj = []
          for sub_child in child.children:
            if sub_child.dep_ in ['nsubj', 'appos', 'acl:relcl'] + can_have_conj:
              sub_child_subj = noun_chunks(sub_child)
              if is_meaning_chunk(doc=doc, chunk=sub_child_subj[0]):
                  all_chunks_subj.append(sub_child_subj)

  return all_chunks_subj

In [10]:
# clause extraction
def is_negative(verb:Token)->bool:
  """
  Check if a verb is negative.
  """
  if verb.lemma_ in ["fail", "unable"]:
    return True
  for child in verb.children:
    if child.dep_ in ['neg'] or child.lemma_ in ['neither', 'without']:
      return True
    for sub_child in child.children:
      if sub_child.dep_ in ['neg']:
        return True
  return False

def is_important(verb:Token)->bool:
  """
  Check if a verb is important
  """
#  if verb.lemma_ not in ['measure', 'characterize', 'report', 'suggest', 'justify', 'demonstrate', 'correspond',
#                         'examine', 'observe']:
  return True

def get_verb_phrase(verb:Token):
  root = verb
  if verb.dep_ not in ['cop']:
    from_verb_dep = ['cop', 'auxpass']
    children = sorted([child for child in verb.children if (child.dep_ in from_verb_dep)] + [verb], key=lambda token: token.idx)
    if len(children) == 1:
      children_string = ' '.join([child.text if child != verb else child.lemma_ for child in children])
    else:
      children_string = ' '.join([child.text if child.dep_ not in ['cop', 'auxpass'] else child.lemma_ for child in children])
  else:
    root = verb.head
    from_noun_dep = ['amod', 'det', 'compound', 'cop', 'case']
    sorted_list_from_root = sorted([c_root for c_root in root.children if c_root.dep_ in from_noun_dep] + [root], key=lambda token: token.idx)
    children_string = ' '.join([child.text if child != verb else child.lemma_ for child in sorted_list_from_root])

  for child in root.children:
    if child.dep_ in ['dobj', 'nmod', 'dep', 'xcomp', 'acl', 'attr']: # acl: used... ; amod: (agent - ROOT) effective in ...
      obj = noun_chunks(noun=child, chunks_for='obj')[0]
      children_string += (' ' + obj)
    elif child.dep_ in ['amod']:
      for amod_child in child.children:
        if amod_child.dep_ in ['nmod', 'xcomp']:
          obj = noun_chunks(noun=amod_child, chunks_for='obj')[0]
          children_string += (' ' + obj)

  return children_string.replace(" 's", "'s")

def find_sub_verbs(verb:Token):
  """
  Find the verbs which relate to a given verb
  """
  if verb.dep_ == 'cop':
    new_verb = verb.head
  else:
    new_verb = verb

  sub_verbs = [s_verb for s_verb in new_verb.children if s_verb.dep_ in ['conj', 'advcl'] and s_verb.pos_ in ['VERB'] and not is_negative(s_verb)]
  idx = 0
  while idx < len(sub_verbs):
    sub_verbs += [s_verb for s_verb in sub_verbs[idx].children if s_verb.dep_ in ['conj'] and s_verb.pos_ in ['VERB'] and not is_negative(s_verb)]
    idx += 1
  return [sub_verb for sub_verb in sub_verbs if is_main_verb(verb=sub_verb) and is_important(verb=sub_verb) and (not is_negative(verb=sub_verb))]

def find_subj_of(verb:Token):
  """
  Find subject for a verb.
  """
  curr_verb = verb
  while True:
    for child in curr_verb.children:
      if child.dep_ in ['nsubj', 'nsubjpass']:
        return child
    if curr_verb == curr_verb.head:
      break
    curr_verb = curr_verb.head

  return None

def is_main_adj(adj:Token):
  """
  Check if an adjective is important or not.
  Example 1: be useful => important
  Example 2: be able to => not important
  """
  for child in adj.children:
    if child.dep_ == 'xcomp' and child.pos_ not in ['NOUN']:
      return False
  
  if adj.pos_ not in ['ADJ']:
    return False
  
  return True

def is_main_verb(verb:Token):
  """
  Check if a verb is important. A verb is not important when it doesn't mean as the function, but the modifier.
  """
  if verb.dep_ in ['amod']:
    return False
  for child in verb.children:
    if child.dep_ == 'xcomp' and child.pos_ == 'VERB':
      return False
  return True

def clause_extraction(nlp:English, sentence:str)->list:
  """
  Extract the clauses in a given sentence.
  Example: A is B and A do C => [A is B ; A do C]
  """
  all_subj = find_all_subj(nlp=nlp, sentence=sentence)
  if len(all_subj) == 0:
    return []

  from_verb_dep = ['det', 'dobj', 'nmod', 'neg']
  clauses = []
  doc = nlp(sentence)
  for subj in all_subj:
    verbs_for_subj = []
    for token in doc:
      if noun_chunks(noun=token) == subj:
        verb = None
        if token.dep_ in ['nsubj', 'nsubjpass']:
          if is_negative(token) or is_negative(token.head):
            continue
          if ((not is_negative(token.head)) and is_important(verb=token.head) and (token.head.pos_ in ['VERB'] and is_main_verb(token.head))) or is_main_adj(token.head):
            verb = token.head
          elif token.head.pos_ in ['NOUN']:
            # connect 2 noun phrase with a cop, such as tobe: A is B ==> A has head is B
            main_result = [child for child in token.head.children if child.pos_ == 'VERB' and child.dep_ == 'cop']
            if len(main_result) > 0:
              verb = main_result[0]

          elif (not is_negative(token.head)) and token.head.pos_ in ['ADJ', 'VERB'] and token.head.dep_ not in ['amod'] and is_important(verb=token.head):
            # Example: The 3-NP administered animals had impaired spatial memory ==> administered is amod
            # but also verb of subj 3-NP
            main_result = [child for child in token.head.children if child.pos_ == 'VERB' and child.dep_ == 'xcomp']
            if len(main_result) > 0:
              verb = main_result[0]
          else:
            continue
        elif token.dep_ in ['appos', 'acl:relcl', 'conj']: # For appos subj 
          # Example: Omega-3, which is a PUFA ==> acl:relcl
          # Example, Omega-3, a fatty acid, ... ==> appos  
           tmp = token

           while tmp.head.pos_ not in ['VERB', 'ADJ'] and tmp != tmp.head: # Find the root of alternative subj if it doesn't has its own verb
             tmp = tmp.head
           
           if (not is_negative(tmp.head) and is_important(verb=tmp.head) and tmp.head.pos_ in ['VERB'] and is_main_verb(tmp.head)) or is_main_adj(tmp.head):
             verb = tmp.head

           elif tmp.head.pos_ in ['NOUN']:
             result = [child for child in tmp.head.children if child.pos_ == 'VERB' and child.dep_ == 'cop']
             if len(result) > 0:
               verb = result[0]
             
           elif (not is_negative(tmp.head)) and tmp.head.pos_ in ['ADJ', 'VERB'] and tmp.head.dep_ not in ['amod'] and is_important(verb=tmp.head):
             result = [child for child in tmp.head.children if child.pos_ == 'VERB' and child.dep_ == 'xcomp']
             if len(result) > 0:
               verb = result[0]
        else:
          continue

        if not verb:
          continue
        
        verbs_for_subj.append(verb)
        sub_verbs = find_sub_verbs(verb=verb)
        if token.dep_ not in ['appos', 'conj', 'acl:relcl']:
          verbs_for_subj += [s_verb for s_verb in sub_verbs if find_subj_of(verb=s_verb) and find_subj_of(verb=s_verb) == token]
        else:
          head_of_subj = token
          while head_of_subj.dep_ not in ['nsubj', 'nsubjpass'] and head_of_subj != head_of_subj.head:
            head_of_subj = head_of_subj.head

          verbs_for_subj += [s_verb for s_verb in sub_verbs if find_subj_of(verb=s_verb) and find_subj_of(verb=s_verb) == head_of_subj]
    for verb in verbs_for_subj:
      verb_phrase = get_verb_phrase(verb=verb)
      if is_meaning_chunk(doc=doc, chunk=verb_phrase):
        clauses.append((subj[0], verb_phrase))
  return list(set(clauses))

In [ ]:
#sentence = "Ascorbic acid has emerged as a neuroprotective compound against neurotoxicants and neurodegenerative diseases, including Alzheimer's disease, multiple sclerosis and amyotrophic lateral sclerosis."
sentence = "Our data provide insight into a mechanism of ErbB4 association with schizophrenia; reveal a previously unidentified biological and disease link between NRG1-ErbB4, p110δ, and AKT; and suggest that p110δ is a previously undescribed therapeutic target for the treatment of psychiatric disorders."
print(sentence)
doc = gena_nlp(process_sentence(sentence=sentence))

for token in doc:
 print(f"{token.text:20} {token.dep_:20} {token.pos_:20} {token.head.text:20} {token.idx:5}")
[print(clause) for clause in clause_extraction(nlp=gena_nlp, sentence=process_sentence(sentence=sentence))]
#print("--------")
#for ent in doc.ents:
#  print(f"{ent.text:40} {ent[-1].dep_:20} {ent[-1].pos_:20}")

Our data provide insight into a mechanism of ErbB4 association with schizophrenia; reveal a previously unidentified biological and disease link between NRG1-ErbB4, p110δ, and AKT; and suggest that p110δ is a previously undescribed therapeutic target for the treatment of psychiatric disorders.
Our                  nmod:poss            PRON                 data                     0
data                 nsubj                NOUN                 provide                  4
provide              ROOT                 VERB                 provide                  9
insight              dobj                 NOUN                 provide                 17
into                 case                 ADP                  mechanism               25
a                    det                  DET                  mechanism               30
mechanism            nmod                 NOUN                 provide                 32
of                   case                 ADP                  association  

[None]

In [ ]:
dep_list = ["ROOT", "acl", "acl:relcl", "acomp", "advcl", "advmod", "amod", "amod@nmod", "appos",
            "attr", "aux", "auxpass", "case", "cc", "cc:preconj", "ccomp", "compound", "compound:prt",
            "conj", "cop", "csubj", "dative", "dep", "det", "det:predet", "dobj", "expl", "intj", "mark",
            "meta", "mwe", "neg", "nmod", "nmod:npmod", "nmod:poss", "nmod:tmod", "nsubj", "nsubjpass", 
            "nummod", "parataxis", "pcomp", "pobj", "preconj", "predet", "prep", "punct", "quantmod", "xcomp"]

for dep in dep_list:
  if spacy.explain(dep):
    print(f"{dep:20} {spacy.explain(dep)}")

acl                  clausal modifier of noun (adjectival clause)
acomp                adjectival complement
advcl                adverbial clause modifier
advmod               adverbial modifier
amod                 adjectival modifier
appos                appositional modifier
attr                 attribute
aux                  auxiliary
auxpass              auxiliary (passive)
case                 case marking
cc                   coordinating conjunction
ccomp                clausal complement
compound             compound
conj                 conjunct
cop                  copula
csubj                clausal subject
dative               dative
dep                  unclassified dependent
det                  determiner
dobj                 direct object
expl                 expletive
intj                 interjection
mark                 marker
meta                 meta modifier
neg                  negation modifier
nmod                 modifier of nominal
nsubj                nomi

In [11]:
# Find max relation
def find_max_relation(relations:list)->tuple:
  """
  From a list of candidate relations, get the longest relation.
  """
  max_length = 0
  max_idx = 0

  idx = 0
  while idx < len(relations):
    if relations[idx][2] - relations[idx][1] > max_length:
      max_length = relations[idx][2] - relations[idx][1]
      max_idx = idx
    idx += 1
  return relations[max_idx]

# Match groups from matcher results
def is_same_group(match_1:tuple, match_2:tuple)->bool:
  if match_1[1] >= match_2[2]:
    return False
  if match_2[1] >= match_1[2]:
    return False
  return True

def group_matches(matches:list)->list:
  groups = []
  if len(matches) > 0:
    curr_longest_match = matches[0]
    for idx in range(1, len(matches)):
      if is_same_group(match_1=matches[idx-1], match_2=matches[idx]):
        if (curr_longest_match[2] - curr_longest_match[1]) < (matches[idx][2] - matches[idx][1]):
          curr_longest_match = matches[idx]
      else:
        groups.append(curr_longest_match)
        curr_longest_match = matches[idx]
    
    # the final match
    groups.append(curr_longest_match)
        
  return groups
  
# noun phrase
def get_noun_phrases(nlp:English, clause:str, pattern:list=NOUN_PHRASE)->list:
  """
  Get all available noun phrases in a given string. 
  """
  np_matcher = Matcher(nlp.vocab)
  np_matcher.add("noun_phrase", [pattern])

  text = nlp(clause)
  np_matches = np_matcher(text)

  groups_np = []
  if len(np_matches) > 0:
    tmp_match = np_matches[0]
    is_same_group = False
    for i in range(len(np_matches)-1):
        if np_matches[i][1] == np_matches[i+1][1] or np_matches[i][2] == np_matches[i+1][2]\
            or tmp_match[1] == np_matches[i+1][1] or tmp_match[2] == np_matches[i+1][2] :
            is_same_group = True
            if (tmp_match[2] - tmp_match[1]) < (np_matches[i+1][2] - np_matches[i+1][1]):
                tmp_match = np_matches[i+1]
        else:
            groups_np.append(tmp_match)
            tmp_match = np_matches[i+1]
        if (i+1) == len(np_matches) - 1:
            groups_np.append(tmp_match)
  return group_matches(np_matches)

# Normalize relation
def simplify_relation(relation:Span)->str:
  """
  Simplify a given relation and convert it into a string.
  """
  new_relation = [token.text for token in relation if token.dep_ not in ['det', 'nummod']]
  return ' '.join(new_relation)

# Relation extraction function
def extract_relation(nlp:English, clause_tuple:tuple)->list:
  """
  Get all ( Noun phrase 01 , relation, Noun phrase 02 ) from an extracted clause tuple (NP1, VP)
  Example: ("A", "do B to get C") => [(A, do, B), (A, do B to get, C)]
  """
  results = []

  subject = clause_tuple[0]
  clause = clause_tuple[1].replace(" 's", "'s")
  relation_phrases = []

  # Verb Phrase
  verb_phrase = [ {'POS': 'VERB', 'OP': '+'}, {'POS': {'IN': ['ADP', 'PART']}, 'OP': '*'}]
  linking_verb_phrase = [ {'POS': 'VERB', 'OP': '+'}, {'POS': 'ADJ', 'OP': '+'}, {'POS': {'IN': ['ADP', 'PART']}, 'OP': '*'}]

  # Added Pattern
  NOUN_PHRASE = [ {'POS': 'DET', 'OP': '?'}, {'POS': 'VERB', 'DEP': {'IN': ['amod', 'acl']}, 'OP': '*'}, {'POS': {'IN': ['ADJ', 'NOUN', 'PRON', 'PROPN', 'NUM', 'PART', 'CCONJ']}, 'OP': '+'}]
  pattern_phrase = verb_phrase + (NOUN_PHRASE + [{'POS': {'IN': ['PART', 'ADP']}}])*3
  relation_phrases.append(pattern_phrase)
  pattern_phrase = verb_phrase + (NOUN_PHRASE + [{'POS': {'IN': ['PART', 'ADP']}}])*2
  relation_phrases.append(pattern_phrase)
  pattern_phrase = verb_phrase + (NOUN_PHRASE + [{'POS': {'IN': ['PART', 'ADP']}}])
  relation_phrases.append(pattern_phrase)

  relation_phrases.append(verb_phrase)

  # Find relation
  text = nlp(clause)
  long_relations = []
  for relation in relation_phrases:
    relation_match = Matcher(nlp.vocab)
    relation_match.add("relation_phrase", [relation])
    relations = relation_match(text)
    if len(relations) > 0:
      max_relation = find_max_relation(relations)
      if is_meaning_chunk(doc=text, chunk=text[max_relation[2]:].text):
        results.append((subject, simplify_relation(text[max_relation[1]:max_relation[2]]).replace(" 's", "'s"), text[max_relation[2]:].text.replace(" 's", "'s")))
      if is_meaning_chunk(doc=text, chunk=text[max_relation[1]:max_relation[2]].text):
        long_relations.append(text[max_relation[1]:max_relation[2]])
      break

  idx = 0
  
  while idx < len(long_relations):
    short_text = long_relations[idx][:-1]
    nb_of_noun_phrase = nb_of_noun_phrase = len(get_noun_phrases(nlp=nlp, clause=short_text.text))
    relation_phrases = []

    pattern_phrase = verb_phrase + (NOUN_PHRASE + [{'POS': {'IN': ['PART', 'ADP']}}])*3
    relation_phrases.append(pattern_phrase)
    pattern_phrase = verb_phrase + (NOUN_PHRASE + [{'POS': {'IN': ['PART', 'ADP']}}])*2
    relation_phrases.append(pattern_phrase)
    pattern_phrase = verb_phrase + (NOUN_PHRASE + [{'POS': {'IN': ['PART', 'ADP']}}])
    relation_phrases.append(pattern_phrase)

    relation_phrases.append(verb_phrase)

    for relation in relation_phrases:
      relation_match = Matcher(nlp.vocab)
      relation_match.add("relation_phrase", [relation])
      
      relations = relation_match(short_text)

      if len(relations) > 0:
        max_relation = find_max_relation(relations)
        if is_meaning_chunk(doc=text, chunk=short_text[max_relation[2]:].text):
          results.append((subject, simplify_relation(relation=short_text[max_relation[1]:max_relation[2]]).replace(" 's", "'s"), short_text[max_relation[2]:].text.replace(" 's", "'s")))
        if is_meaning_chunk(doc=text, chunk=short_text[max_relation[1]:max_relation[2]].text):
          long_relations.append(short_text[max_relation[1]:max_relation[2]])
        break
    idx += 1

  return results

In [ ]:
sentence = "Pioglitazone (or glitazone) ameliorated LPS-induced dopaminergic neuronal loss, as well as mitigated neurobehavioral impairments while  vitamin C administration successfully attenuated memory impairment, fatigue development and more."
sentence_processed = process_sentence(sentence=sentence)
for token in gena_nlp(sentence_processed):
  print(f"{token.text:20} {token.head.text:20} {token.dep_:20} {token.pos_}")

clauses = clause_extraction(nlp=gena_nlp, sentence=sentence_processed)
if len(clauses) > 0:
  for clause in clauses:
    print(extract_relation(nlp=gena_nlp, clause_tuple=clause))

Pioglitazone         ameliorated          nsubj                NOUN
(                    Pioglitazone         punct                PUNCT
or                   Pioglitazone         cc                   CCONJ
glitazone            Pioglitazone         conj                 NOUN
)                    Pioglitazone         punct                PUNCT
ameliorated          ameliorated          ROOT                 VERB
LPS-induced          loss                 amod                 ADJ
dopaminergic         loss                 amod                 ADJ
neuronal             loss                 amod                 ADJ
loss                 ameliorated          dobj                 NOUN
,                    loss                 punct                PUNCT
as                   ameliorated          cc                   ADV
well                 as                   mwe                  ADV
as                   as                   mwe                  ADP
mitigated            ameliorated          conj    

---

## Relation Extraction Function

Apply `extract_relation` function to any sentence which has at least a valid clause. 

In [ ]:
sentences = pd.read_csv("sentences.csv")['Sentence'].tolist()
no_clause = 0
no_relation = 0
has_relation = 0

for sent in sentences[:200]:
  if is_figured_sentence(sent):
    continue
  sentence = process_sentence(sentence=sent)
  print(sentence)
  print()
  clauses = clause_extraction(nlp=gena_nlp, sentence=sentence)
  if len(clauses) > 0:
    for clause in clauses:
      relations = extract_relation(nlp=gena_nlp, clause_tuple=clause)
      if len(relations) > 0:
        for relation in relations:
          print(relation)
          has_relation += 1
        print()
      else:
        no_relation += 1
  else:
    no_clause += 1
  print("-----")
  print()

print(f"There are {no_clause} sentences has no clause and {no_relation} sentences has no relation.")
print(f"There are {has_relation} relations in {len(sentences[:200])} sentences.")

In humans, until today, only two case reports are available reporting successful treatment with dronabinol (tetrahydrocannabinol, THC), an agonist at central cannabinoid CB1 receptors, in patients with otherwise treatment refractory OCD.

-----

These reports are in line with data from a limited number of case studies and small controlled trials in patients with Tourette syndrome (TS), a chronic motor and vocal tic disorder often associated with comorbid obsessive compulsive behavior (OCB), reporting an improvement of tics and of comorbid OCB after use of different kinds of cannabis-based medicines including THC, cannabis extracts, and flowers.

-----

Here we present the case of a 22-year-old male patient, who suffered from severe OCD since childhood and significantly improved after treatment with medicinal cannabis with markedly reduced OCD and depression resulting in a considerable improvement of quality of life.

-----

In TS/CTD, RA may be regarded as a major comorbidity that requ

---

## Non-verb Relation Extraction

Apply some common patterns to extract the relation between entities that are not connected by a verb.

In [ ]:
sentence = "Also, compromised contextual and cued fear conditioning in these animals suggests a considerable memory deficit, thus potentially forming a connection to the previously determined ether lipid deficit in human patients with Alzheimer's disease."
doc = gena_nlp(sentence)
for token in doc:
  print(f"{token.text:20} {token.dep_:20} {token.pos_:20} {token.head.text}")

Also                 advmod               ADV                  compromised
,                    punct                PUNCT                compromised
compromised          ROOT                 VERB                 compromised
contextual           dobj                 NOUN                 compromised
and                  cc                   CCONJ                compromised
cued                 conj                 VERB                 compromised
fear                 compound             NOUN                 conditioning
conditioning         dobj                 NOUN                 cued
in                   case                 ADP                  animals
these                det                  DET                  animals
animals              nmod                 NOUN                 conditioning
suggests             conj                 VERB                 compromised
a                    det                  DET                  deficit
considerable         amod                 

In [12]:
def triple_generate_from_pattern(nlp:English, text:Doc, groups_matches:list, type_triple:str="left", pattern=NOUN_PHRASE)->tuple:
  """
  Generate triple from a group matches which contains the no-verb patterns.
  """
  for group in groups_matches:    
    no_verb_text = text[group[1]:group[2]]
    noun_phrases = [(no_verb_text[np[1]-1], no_verb_text[np[1]:np[2]]) for np in get_noun_phrases(nlp=nlp, clause=no_verb_text.text, pattern=pattern)]
    meaning_noun_phrases = [np for np in noun_phrases if is_meaning_chunk(doc=text, chunk=np[1].text)]
    
    if len(meaning_noun_phrases) >= 2 and len(noun_phrases) == 3:
      if type_triple == 'left':
        np_1 = noun_phrases[1][1].text
        np_2 = noun_phrases[2][1].text
        relation = "have " + noun_phrases[0][1].text + " " + noun_phrases[2][0].text
      elif type_triple == 'right':
        np_1 = noun_phrases[2][1].text
        np_2 = noun_phrases[1][1].text
        relation = "have " + noun_phrases[0][1].text + " " + noun_phrases[1][0].text
      elif type_triple == 'center':
        np_1 = noun_phrases[1][1].text
        np_2 = noun_phrases[2][1].text
        relation = "have " + noun_phrases[0][1].text + " with"
        if is_meaning_chunk(doc=text, chunk=np_1) and is_meaning_chunk(doc=text, chunk=np_2):
          return ((np_1, relation, np_2), (np_2, relation, np_1))
      else:
        np_1 = noun_phrases[0][1].text
        np_2 = noun_phrases[2][1].text
        relation = noun_phrases[1][0].text + " " + noun_phrases[1][1].text + " " + noun_phrases[2][0].text

      if is_meaning_chunk(doc=text, chunk=np_1) and is_meaning_chunk(doc=text, chunk=np_2):        
        return (np_1, relation, np_2)

    return None

def non_verb_relation_extraction(nlp:English, sentence:str)->list:
  """
  Extract relations between no-verb connected noun-phrases
  """
  text = nlp(sentence)
  results = []

  # Left -> Right
  pattern = NOUN_PHRASE + [ {'LOWER': {'IN': ['in', 'of']}} ] + NOUN_PHRASE + [ {'LOWER': {'IN': ['to', 'on', 'for', 'in', 'with']}} ] + NOUN_PHRASE
  # Left -> Right Pattern Matching
  no_verb_matcher = Matcher(nlp.vocab)
  no_verb_matcher.add("no_verb_matcher", [pattern])

  matches = no_verb_matcher(text)
  groups = group_matches(matches=matches)
  result = triple_generate_from_pattern(nlp=nlp, text=text, groups_matches=groups)
  if result:
    results.append(result)

  # Right -> Left
  pattern = NOUN_PHRASE + [ {'LOWER': {'IN': ['to', 'for', 'of', 'on']}} ] + NOUN_PHRASE + [ {'LOWER': {'IN': ['by', 'through']}} ] + NOUN_PHRASE
  # Right -> Left Pattern Matching
  no_verb_matcher = Matcher(nlp.vocab)
  no_verb_matcher.add("no_verb_matcher", [pattern])

  matches = no_verb_matcher(text)
  groups = group_matches(matches=matches)
  result = triple_generate_from_pattern(nlp=nlp, text=text, groups_matches=groups, type_triple="right")
  if result:
    results.append(result)

  # Left -> Right and Left <- Right
  pattern = NO_CONJ_NOUN_PHRASE + [{'LOWER': 'between'}] + NO_CONJ_NOUN_PHRASE + [{'LOWER': 'and'}] + NO_CONJ_NOUN_PHRASE

  # Center Pattern Matching
  no_verb_matcher = Matcher(nlp.vocab)
  no_verb_matcher.add("no_verb_matcher", [pattern])

  matches = no_verb_matcher(text)
  groups = group_matches(matches=matches)
  result = triple_generate_from_pattern(nlp=nlp, text=text, groups_matches=groups, type_triple="center", pattern=NO_CONJ_NOUN_PHRASE)
  if result:
    results += result

  # Observation Pattern
  pattern = NO_CONJ_NOUN_PHRASE + [{'LOWER': 'in'}, {'POS': {'IN': ['NOUN', 'ADJ']}, 'OP': '*'}, {'LOWER': {'IN': ['patient', 'patients', 'mice', 'rat', 'rats', 'mouse', 'animal', 'animals']}}, {'LOWER': 'with'}] + NO_CONJ_NOUN_PHRASE
  
  # Observation Pattern Matching
  no_verb_matcher = Matcher(nlp.vocab)
  no_verb_matcher.add("no_verb_matcher", [pattern])

  matches = no_verb_matcher(text)
  groups = group_matches(matches=matches)
  
  result = triple_generate_from_pattern(nlp=nlp, text=text, groups_matches=groups, type_triple="observation", pattern=NO_CONJ_NOUN_PHRASE)
  if result:
    results.append(result)

  return results

In [ ]:
sentences = ["The hypothesis considered states that low omega-3 PUFA abundance in schizophrenia patients with chronic stress and related disorders is due to an underlying genetically determined abnormality.",
             "In this study, we investigated the role of DHA in stress-evoked pain chronicity using diet-induced n-3 fatty acid deficient mice.",
             "The robust relationship between low erythrocyte n-3 PUFA concentrations and SAD justifies exploration of relevant neuropathophysiological mechanisms."]
for sentence in sentences:
  print(sentence)
  print(non_verb_relation_extraction(nlp=gena_nlp, sentence=sentence))
  print('---')

The hypothesis considered states that low omega-3 PUFA abundance in schizophrenia patients with chronic stress and related disorders is due to an underlying genetically determined abnormality.
[('schizophrenia patients', 'have low omega-3 PUFA abundance with', 'chronic stress and related disorders'), ('low omega-3 PUFA abundance', 'in schizophrenia patients with', 'chronic stress')]
---
In this study, we investigated the role of DHA in stress-evoked pain chronicity using diet-induced n-3 fatty acid deficient mice.
[('DHA', 'have the role in', 'stress-evoked pain chronicity')]
---
The robust relationship between low erythrocyte n-3 PUFA concentrations and SAD justifies exploration of relevant neuropathophysiological mechanisms.
[('low erythrocyte n-3 PUFA concentrations', 'have The robust relationship with', 'SAD'), ('SAD', 'have The robust relationship with', 'low erythrocyte n-3 PUFA concentrations')]
---


In [ ]:
sentences = pd.read_csv("sentences.csv")['Sentence'].tolist()
no_relation = 0

for sent in sentences[:200]:
  if is_figured_sentence(sent):
    continue
  sentence = process_sentence(sentence=sent)
  print(sentence)
  print()

  # Find relation within a verb
  clauses = clause_extraction(nlp=gena_nlp, sentence=sentence)
  if len(clauses) > 0:
    for clause in clauses:
      relations = extract_relation(nlp=gena_nlp, clause_tuple=clause)
      if len(relations) > 0:
        for relation in relations:
          print(relation)
        print()
  
  # Find relation without any verb
  no_verb_triples = non_verb_relation_extraction(nlp=gena_nlp, sentence=sentence)
  if len(no_verb_triples) > 0:
    for triple in no_verb_triples:
      print(triple)
    print()
  
  # Cannot find any relation
  if len(clauses) == 0 and len(no_verb_triples) == 0:
    no_relation += 1

  print("-----")
  print()

print(f"There are {no_relation} sentences has no relation.")

In humans, until today, only two case reports are available reporting successful treatment with dronabinol (tetrahydrocannabinol, THC), an agonist at central cannabinoid CB1 receptors, in patients with otherwise treatment refractory OCD.

-----

These reports are in line with data from a limited number of case studies and small controlled trials in patients with Tourette syndrome (TS), a chronic motor and vocal tic disorder often associated with comorbid obsessive compulsive behavior (OCB), reporting an improvement of tics and of comorbid OCB after use of different kinds of cannabis-based medicines including THC, cannabis extracts, and flowers.

-----

Here we present the case of a 22-year-old male patient, who suffered from severe OCD since childhood and significantly improved after treatment with medicinal cannabis with markedly reduced OCD and depression resulting in a considerable improvement of quality of life.

-----

In TS/CTD, RA may be regarded as a major comorbidity that requ

-----

## Triples of Entity Construction

In [13]:
def full_entities(entity:Token):
  full_ent = ''
  sorted_token = sorted([child for child in entity.children if (child.dep_ in ['amod', 'compound', 'nmod:poss']) and (child.lemma_ not in ['his', 'her', 'your', 'their', 'my', 'its', 'our'])] + [entity], key=lambda token:token.idx)
  return ' '.join([t.text if (t.dep_ != 'nmod:poss') else (t.text + '\'s') for t in sorted_token])

def get_entities(noun_phrase:Doc):
  """
  Get named entities in a noun phrase.
  """
  entities_label = ['CHEMICAL', 'CHEBI', 'NUTRITION', 'DISEASE', 'MENTAL_HEALTH']
  full_entity = ['appos', 'acl:recl', 'nmod', 'dobj', 'conj', 'dep', 'ROOT']
  supplement_entity = ['amod', 'compound']
  
  entities = []
  for ent in noun_phrase.ents:
    if ent.label_ in entities_label:
      if ent[-1].dep_ in full_entity:
        entities.append((full_entities(ent[-1]), ent.label_))
      elif ent[-1].dep_ in supplement_entity:
        entities.append((full_entities(ent[-1].head), ent.label_))
  return entities

def extract_entities_relation(nlp:English, triple:tuple):
  """
  Extract entity relation triples from a given noun phrase relation triple.
  """
  entities_1 = []
  entities_2 = []
  
  # Get entities from noun phrases group 1
  np_1 = triple[0]
  doc_1 = nlp(np_1)
  entities_1 = get_entities(doc_1)

  # Get entities from noun phrases group 2
  np_2 = triple[2]
  doc_2 = nlp(np_2)
  entities_2 = get_entities(doc_2)

  # Get only unique values of entities_1 and entities_2
  entities_1 = set(entities_1)
  entities_2 = set(entities_2)

  # Generate entities_triple
  entity_triples = []
  for ent_1 in entities_1:
    for ent_2 in entities_2:
      new_triple = (ent_1[0], ent_1[1], triple[1], ent_2[0], ent_2[1])
      entity_triples.append(new_triple)

  return entity_triples

In [ ]:
sentences = pd.read_csv("sentences.csv")['Sentence'].tolist()
no_relation = 0
entitiy_triples = []
for sent in sentences[:200]:
  if is_figured_sentence(sent):
    continue
  sentence = process_sentence(sentence=sent)
  print(sentence)

  # Find relation within a verb
  clauses = clause_extraction(nlp=gena_nlp, sentence=sentence)
  if len(clauses) > 0:
    for clause in clauses:
      relations = extract_relation(nlp=gena_nlp, clause_tuple=clause)
      if len(relations) > 0:
        for relation in relations:
          entity_triple = extract_entities_relation(nlp=gena_nlp, triple=relation)
          if len(entity_triple) > 0:
            for triple in entity_triple:
              entitiy_triples.append((sentence, triple[0], triple[1], triple[2]))
              print(triple)
  
  # Find relation without any verb
  no_verb_triples = non_verb_relation_extraction(nlp=gena_nlp, sentence=sentence)
  if len(no_verb_triples) > 0:
    for triple in no_verb_triples:
      entity_triple = extract_entities_relation(nlp=gena_nlp, triple=triple)
      if len(entity_triple) > 0:
        for triple in entity_triple:
          entitiy_triples.append((sentence, triple[0], triple[1], triple[2]))
          print(triple)
  
  # Cannot find any relation
  if len(clauses) == 0 and len(no_verb_triples) == 0:
    no_relation += 1
  print("---")
  
print(f"There are {no_relation} sentences has no relation.")


In humans, until today, only two case reports are available reporting successful treatment with dronabinol (tetrahydrocannabinol, THC), an agonist at central cannabinoid CB1 receptors, in patients with otherwise treatment refractory OCD.
---
These reports are in line with data from a limited number of case studies and small controlled trials in patients with Tourette syndrome (TS), a chronic motor and vocal tic disorder often associated with comorbid obsessive compulsive behavior (OCB), reporting an improvement of tics and of comorbid OCB after use of different kinds of cannabis-based medicines including THC, cannabis extracts, and flowers.
---
Here we present the case of a 22-year-old male patient, who suffered from severe OCD since childhood and significantly improved after treatment with medicinal cannabis with markedly reduced OCD and depression resulting in a considerable improvement of quality of life.
---
In TS/CTD, RA may be regarded as a major comorbidity that requires clinica

# Implement for the entire sentence dataset

In [14]:
sentences_df = pd.read_csv("sentences_3.csv")

entity_triples = []

for (idx, row) in sentences_df.iterrows():
  print('Sentence', idx)
  sentence = process_sentence(sentence=row['Sentence'])
  print(sentence)

  # Find relation within a verb
  clauses = clause_extraction(nlp=gena_nlp, sentence=sentence)
  if len(clauses) > 0:
    for clause in clauses:
      relations = extract_relation(nlp=gena_nlp, clause_tuple=clause)
      if len(relations) > 0:
        for relation in relations:
          entity_triple = extract_entities_relation(nlp=gena_nlp, triple=relation)
          if len(entity_triple) > 0:
            for triple in entity_triple:
              entity_triples.append((row['PMID'], row['Sentence'], *triple))
  
  # Find relation without any verb
  no_verb_triples = non_verb_relation_extraction(nlp=gena_nlp, sentence=sentence)
  if len(no_verb_triples) > 0:
    for no_verb_triple in no_verb_triples:
      entity_triple = extract_entities_relation(nlp=gena_nlp, triple=no_verb_triple)
      if len(entity_triple) > 0:
        for triple in entity_triple:
          entity_triples.append((row['PMID'], row['Sentence'], *triple))

Streaming output truncated to the last 5000 lines.
Sentence 8600
Soy isoflavones, found in soybean and soybean products, have been reported to possess many physiological activities such as antioxidant activity, inhibition of cancer cell proliferation, reduction of cardiovascular risk, prevention of osteoporosis and alleviation of postmenopausal syndrome.
Sentence 8601
In our previous study, soy isoflavone extract ISO-1 (containing 12 soy isoflavones) from soybean cake was demonstrated to prevent skin damage caused by UVB exposure.
Sentence 8602
In this study, soy isoflavone extract from soybean cake was further purified and evaluated for the protective effects on UVB-induced damage.
Sentence 8603
The results revealed that Fraction 3, which contains the aglycone group (daidzein, genistein and glycitein) and acetylglucoside group (acetyldaidzin, acetylgenistin and acetylglycitin) of soy isoflavones, could inhibit UVB-induced death of human keratinocytes and reduce the level of desquamati

In [15]:
print(f"There are {len(entity_triples)} relations between 2 entities in all sentences.")

There are 8692 relations between 2 entities in all sentences.


In [16]:
entities_relations_all_df = pd.DataFrame(entity_triples, columns=["PMID", "Sentence", "E1", "Type_E1", "Relation", "E2", "Type_E2"])
entities_relations_all_df.head(5)

,PMID,Sentence,E1,Type_E1,Relation,E2,Type_E2
0,28470822,Transforming growth factor beta 1 (TGFB1) is d...,Transforming growth factor beta,CHEBI,be dysregulated in,subcortical infarcts,DISEASE
1,28470822,Transforming growth factor beta 1 (TGFB1) is d...,Transforming growth factor beta,CHEBI,be dysregulated in,cerebral autosomal recessive,DISEASE
2,28470822,Transforming growth factor beta 1 (TGFB1) is d...,Transforming growth factor beta,CHEBI,be dysregulated in,hereditary SVD,DISEASE
3,28470822,Transforming growth factor beta 1 (TGFB1) is d...,Transforming growth factor beta,CHEBI,be dysregulated in,recessive arteriopathy,DISEASE
4,28470822,Transforming growth factor beta 1 (TGFB1) is d...,Transforming growth factor beta,CHEBI,be dysregulated in,leukoencephalopathy,DISEASE


In [17]:
remove_regex = r"such | such"
entities_relations_all_df['E1'] = entities_relations_all_df['E1'].apply(lambda e: re.sub(remove_regex, "", str(e)).strip())
entities_relations_all_df['E2'] = entities_relations_all_df['E2'].apply(lambda e: re.sub(remove_regex, "", str(e)).strip())
entities_relations_all_df['Relation'] = entities_relations_all_df['Relation'].apply(lambda r: (r.lower()))

In [18]:
entities_relations_all_df.to_csv("gena_data_raw_3.csv", index=False)

------

# Example

In [ ]:
sentence = "Pioglitazone (or glitazone) ameliorated LPS-induced dopaminergic neuronal loss, as well as mitigated neurobehavioral impairments while  vitamin C administration successfully attenuated memory impairment, fatigue development and more."
sentence_processed = process_sentence(sentence=sentence)

clauses = clause_extraction(nlp=gena_nlp, sentence=sentence_processed)

for clause in clauses:
  relations = extract_relation(nlp=gena_nlp, clause_tuple=clause)
  if len(relations) > 0:
    print("Clause:", clause)
    [print("Relation:", relation) for relation in relations]
    for relation in relations:
      entity_triple = extract_entities_relation(nlp=gena_nlp, triple=relation)
      if len(entity_triple) > 0:
        for triple in entity_triple:
          print("Triple:", triple)
    print("---")

Clause: ('glitazone', 'mitigate neurobehavioral impairments')
Relation: ('glitazone', 'mitigate', 'neurobehavioral impairments')
Triple: ('glitazone', 'CHEMICAL', 'mitigate', 'neurobehavioral impairments', 'DISEASE')
---
Clause: ('Pioglitazone', 'mitigate neurobehavioral impairments')
Relation: ('Pioglitazone', 'mitigate', 'neurobehavioral impairments')
Triple: ('Pioglitazone', 'CHEMICAL', 'mitigate', 'neurobehavioral impairments', 'DISEASE')
---
Clause: ('Pioglitazone', 'ameliorate LPS-induced dopaminergic neuronal loss ,')
Relation: ('Pioglitazone', 'ameliorate', 'LPS-induced dopaminergic neuronal loss ,')
Triple: ('Pioglitazone', 'CHEMICAL', 'ameliorate', 'LPS-induced dopaminergic neuronal loss', 'DISEASE')
---
Clause: ('vitamin C administration', 'attenuate memory impairment , fatigue development and more')
Relation: ('vitamin C administration', 'attenuate', 'memory impairment , fatigue development and more')
Triple: ('vitamin C administration', 'CHEBI', 'attenuate', 'memory impair

In [ ]:
#sentence = "Furthermore, PDE5-Is are currently under various phases of clinical or preclinical researches with promising potential for other urinary and genital illnesses, such as priapism, premature ejaculation, urinary tract calculi, overactive bladder, Peyronie's disease, and female sexual dysfunction."
# sentence = "We previously reported that a certain subtype of schizophrenic patients exhibit idiopathic carbonyl stress with high plasma pentosidine levels and the depletion of vitamin B6, without underlying diabetes or chronic kidney disease, the two major causes of elevated AGEs."
sentence = "Flibanserin, a multifunctional serotonin receptor agonist and antagonist, is currently approved in the United States and Canada for the treatment of acquired, generalized hypoactive sexual desire disorder (HSDD) in premenopausal women."
clauses = clause_extraction(nlp=gena_nlp, sentence=sentence)
for clause in clauses:
  relations = extract_relation(nlp=gena_nlp, clause_tuple=clause)
  for relation in relations:
    print(relation)
    entity_triple = extract_entities_relation(nlp=gena_nlp, triple=relation)
    if len(entity_triple) > 0:
      for triple in entity_triple:
        print("Triple:", triple)
        print('---')

('multifunctional serotonin receptor agonist', 'be approved in United States and Canada for treatment of', 'acquired , generalized hypoactive sexual desire disorder ( HSDD ) in premenopausal women')
Triple: ('multifunctional serotonin receptor agonist', 'CHEBI', 'be approved in United States and Canada for treatment of', 'acquired generalized hypoactive sexual desire disorder', 'MENTAL_HEALTH')
---
Triple: ('multifunctional serotonin receptor agonist', 'CHEBI', 'be approved in United States and Canada for treatment of', 'HSDD', 'DISEASE')
---
Triple: ('multifunctional serotonin receptor agonist', 'CHEMICAL', 'be approved in United States and Canada for treatment of', 'acquired generalized hypoactive sexual desire disorder', 'MENTAL_HEALTH')
---
Triple: ('multifunctional serotonin receptor agonist', 'CHEMICAL', 'be approved in United States and Canada for treatment of', 'HSDD', 'DISEASE')
---
('Flibanserin', 'be approved in United States and Canada for treatment of', 'acquired , general

In [ ]:
sentence = "Vitamin C (ascorbic acid) and vitamin E, which is a fat-soluble nutrient, have beneficials effects on psychotic disorder and improve anxiety"
sentence_processed = process_sentence(sentence=sentence)
clauses = clause_extraction(nlp=gena_nlp, sentence=sentence_processed)
for clause in clauses:
  relations = extract_relation(nlp=gena_nlp, clause_tuple=clause)
  if len(relations) > 0:
    [print("Relation:", relation) for relation in relations]
    for relation in relations:
      entity_triple = extract_entities_relation(nlp=gena_nlp, triple=relation)
      if len(entity_triple) > 0:
        for triple in entity_triple:
          print("Triple:", triple)
    print("---")

Relation: ('ascorbic acid', 'have beneficials effects on', 'psychotic disorder')
Triple: ('ascorbic acid', 'CHEBI', 'have beneficials effects on', 'psychotic disorder', 'MENTAL_HEALTH')
---
Relation: ('fat-soluble nutrient', 'have beneficials effects on', 'psychotic disorder')
Triple: ('fat-soluble nutrient', 'CHEMICAL', 'have beneficials effects on', 'psychotic disorder', 'MENTAL_HEALTH')
---
Relation: ('Vitamin C', 'have beneficials effects on', 'psychotic disorder')
Triple: ('Vitamin C', 'CHEBI', 'have beneficials effects on', 'psychotic disorder', 'MENTAL_HEALTH')
---
Relation: ('Vitamin C', 'improve', 'anxiety')
Triple: ('Vitamin C', 'CHEBI', 'improve', 'anxiety', 'MENTAL_HEALTH')
---
Relation: ('fat-soluble nutrient', 'improve', 'anxiety')
Triple: ('fat-soluble nutrient', 'CHEMICAL', 'improve', 'anxiety', 'MENTAL_HEALTH')
---
Relation: ('vitamin E', 'improve', 'anxiety')
Triple: ('vitamin E', 'CHEBI', 'improve', 'anxiety', 'MENTAL_HEALTH')
---
Relation: ('ascorbic acid', 'improve